## Constants

In [61]:
import os

file_count = 50   # TODO: change to 1000
playlists_per_file = 1000

dir_data_raw = os.path.join("..", "data", "raw")
dir_data_processed = os.path.join("..", "data", "processed")

playlist_count = file_count * playlists_per_file

Merge the data extracted from the Spotify API (csv file) with that of the Spotify Million Playlist Dataset (pickle file)

In [62]:
import os
import pandas as pd

path_audio_features = os.path.join(dir_data_raw, "tracks", "track_audio_features.csv")
audio_features = pd.read_csv(path_audio_features).set_index("id")

display(audio_features)

,tempo,key,mode,loudness,danceability,energy,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,time_signature
id,,,,,,,,,,,,,
0000uJA4xCdxThagdLkkLR,184.913,5.0,1.0,-5.621,0.458,0.5910,0.0326,0.5680,0.000015,0.2860,0.6540,161187.0,3.0
0002yNGLtYSYtc0X6ZnFvp,182.345,8.0,1.0,-11.572,0.455,0.6230,0.0523,0.7970,0.903000,0.6340,0.9510,220293.0,4.0
00039MgrmLoIzSpuYKurn9,132.064,1.0,1.0,-5.632,0.742,0.7530,0.0364,0.0178,0.000000,0.1330,0.2630,222727.0,4.0
0005rgjsSeVLp1cze57jIN,133.158,1.0,0.0,-6.141,0.507,0.4460,0.0276,0.7990,0.000000,0.3190,0.4180,213960.0,4.0
0006Rv1e2Xfh6QooyKJqKS,89.048,2.0,0.0,-9.190,0.295,0.4980,0.0301,0.7950,0.944000,0.1070,0.0445,189639.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7zzptITgTKf4HpJM8ye47v,81.071,1.0,0.0,-6.398,0.447,0.7240,0.0372,0.7880,0.202000,0.2420,0.9400,215813.0,4.0
7zzpwV2lgKsLke68yFoZdp,129.996,7.0,1.0,-2.558,0.497,0.6980,0.0317,0.1270,0.000000,0.1160,0.5520,233933.0,4.0
7zzrzbrb14URUZlmSrCGfM,98.463,9.0,0.0,-25.031,0.347,0.0127,0.0453,0.9490,0.000033,0.0478,0.1510,340560.0,3.0


Create a function to compute relevant metrics about a playlist's audio features.

In [63]:
import sys

def get_playlist_metrics(df: pd.core.frame.DataFrame):
    # print("Inside get_playlist_metrics")

    target_columns = list(range(2,5)) # TODO: which columns do we care about?
    def get_statistic(f) -> pd.core.frame.DataFrame:
        return df.apply(f).to_frame().transpose()

    assert isinstance(df, pd.core.frame.DataFrame)
    result = pd.concat({"Mean" : get_statistic(pd.Series.mean),
                      "Median" : get_statistic(pd.Series.median),
                      "Standard deviation" : get_statistic(pd.Series.std),
                      "Variance" : get_statistic(pd.Series.var)}, axis=1)
    
    result.columns = result.columns.swaplevel(0, 1)
    result.sort_index(axis=1, level=0, inplace=True)
    return result

Combine extracted and calculated data into a playlist frame

In [64]:
import pickle

playlist_list_path = os.path.join(dir_data_processed, "playlists" + str(playlist_count) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

playlists_metrics = pd.DataFrame()

total_processed = 0 #TODO: DELETE ME WHEN FINISHED TESTING!
total_tracks = 0 #TODO: DELETE ME WHEN FINISHED TESTING!

for i, playlist in enumerate(playlist_data[5:100]):
    track_counter = 0 # TODO: DELETE ME WHEN FINISHED TESTING!
    playlist_features = pd.DataFrame(columns = audio_features.columns.tolist())

    for track_id in playlist["track_ids"]:
        total_tracks += 1

        if track_id not in audio_features.index: # TODO: handle cases when track id has no audio features
            continue

        track_features = pd.DataFrame(audio_features.loc[track_id]).T # .loc gives series, so transpose is needed
        # display(track_features)

        track_counter += 1
        total_processed += 1
        
        playlist_features = pd.concat([playlist_features if not playlist_features.empty else None, 
                                       track_features], axis=0)
        
    # print("Playlist features")
    # display(playlist_features)

    # print("{:3} - Processed tracks:{:3}/{:3}".format(i, track_counter, len(playlist["track_ids"])))    
    
    # get playlist meta data:
    playlist_info = pd.DataFrame(data={k:[v] for k,v in playlist.items() if k != "track_ids"})
    columns_group = pd.MultiIndex.from_product([['Playlist Data'], playlist_info.columns])
    playlist_info.columns = columns_group
    # display(playlist_info)
    
    
    # print("Calling get_playlist_metrics")
    # display(get_playlist_metrics(playlist_features)) ###############
    metrics = get_playlist_metrics(playlist_features)
    # display(metrics)
    playlist_metrics = pd.concat([playlist_info, metrics], axis=1)
    # display(playlist_metrics)
    playlists_metrics = pd.concat([playlists_metrics, playlist_metrics], ignore_index=True)
    # break

display(playlists_metrics)
print("{} Playlists | Total processed tracks: {}/{}".format(i+1, total_processed, total_tracks))

Playlist Data                                                       \
              name collaborative  pid modified_at num_tracks num_albums   
0           run it         false   25  1488412800         41         38   
1             Yeet         false   28  1509062400         81         57   
2             2020         false   40  1490659200         85         80   
3    Road Trippin'         false   42  1490572800        158        131   
4     taylor swift         false   44  1430611200         33         29   
..             ...           ...  ...         ...        ...        ...   
90            spin         false  382  1505347200         14         14   
91    country 2017         false  383  1505606400         31         30   
92  *** good stuff         false  384  1507593600         46         43   
93              mb         false  389  1509062400        174        161   
94          tears.         false  391  1509148800         68         56   

                                       acousticness  ...        tempo  \
   num_followers num_edits num_artists         Mean  ...     Variance   
0              2         8          31     0.113380  ...   516.504084   
1              7        26          37     0.130906  ...   825.276472   
2              6        27          71     0.219232  ...   953.079950   
3              2        51         106     0.196531  ...   833.555717   
4              3        18          21     0.254192  ...   782.880000   
..           ...       ...         ...          ...  ...          ...   
90             2         5          13     0.041466  ...   133.778640   
91             2        17          29     0.120373  ...  1009.142030   
92             2        18          37     0.399530  ...   718.645004   
93             2        81         140     0.223396  ...   882.318251   
94             3        18          47     0.611977  ...   889.595386   

   time_signature                                       valence          \
             Mean Median Standard deviation  Variance      Mean  Median   
0        4.000000    4.0           0.235702  0.055556  0.509514  0.4880   
1        4.052632    4.0           0.224781  0.050526  0.471933  0.4700   
2        3.986486    4.0           0.350860  0.123103  0.469955  0.4860   
3        4.000000    4.0           0.170872  0.029197  0.558068  0.5790   
4        3.965517    4.0           0.325440  0.105911  0.489241  0.5200   
..            ...    ...                ...       ...       ...     ...   
90       4.000000    4.0           0.000000  0.000000  0.582917  0.6215   
91       4.000000    4.0           0.000000  0.000000  0.641034  0.6570   
92       3.880952    4.0           0.395239  0.156214  0.389079  0.3320   
93       3.981250    4.0           0.262352  0.068829  0.451137  0.4665   
94       3.966102    4.0           0.260378  0.067797  0.282969  0.2680   

                                Playlist Data  
   Standard deviation  Variance   description  
0            0.223534  0.049968           NaN  
1            0.174424  0.030424           NaN  
2            0.206586  0.042678           NaN  
3            0.224833  0.050550           NaN  
4            0.230840  0.053287           NaN  
..                ...       ...           ...  
90           0.233148  0.054358           NaN  
91           0.204880  0.041976           NaN  
92           0.217971  0.047511           NaN  
93           0.207410  0.043019           NaN  
94           0.177707  0.031580           NaN  

[95 rows x 62 columns]

95 Playlists | Total processed tracks: 6966/7675


In [65]:
import pickle

playlist_list_path = os.path.join(dir_data_processed, "playlists" + str(playlist_count) + ".pkl")

with open(playlist_list_path, "rb") as fin:
    playlist_data = pickle.load(fin)

playlist_frame
playlist_frame = pd.DataFrame(np.arrange())

NameError: name 'playlist_frame' is not defined